# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [108]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [169]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=hy530370")
cur = conn.cursor()

In [110]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [111]:
song_files = get_files('data/song_data')
song_files

['/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAADZ128F9348C2E

In [124]:
filepath = song_files[0]
filepath

'/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json'

In [150]:
from pandas import json_normalize
import json

df = pd.DataFrame()
for song_file in song_files:
    with open(song_file,'r') as f:
        json_data = json.load(f)
        song_data = json_normalize(json_data)
        df = pd.concat([df,song_data],axis=0,ignore_index=True)
df.head()


# with open(filepath,'r') as f:
#     json_data = json.load(f)

#     song_data = json_normalize(json_data)
# type(song_data)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
1,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
2,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
3,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
4,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969


In [152]:
for value in df.head().values:
    print(type(value))
    num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = value
    artist_data = (artist_id, artist_name, artist_location, artist_latitude, artist_longitude)
    print(type(artist_data))
    print(artist_data)


<class 'numpy.ndarray'>
<class 'tuple'>
('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan)
<class 'numpy.ndarray'>
<class 'tuple'>
('AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan)
<class 'numpy.ndarray'>
<class 'tuple'>
('ARXR32B1187FB57099', 'Gob', '', nan, nan)
<class 'numpy.ndarray'>
<class 'tuple'>
('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', nan, nan)
<class 'numpy.ndarray'>
<class 'tuple'>
('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892)


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [139]:
song_table_data = json_normalize(json_data)[['song_id','title','artist_id','year','duration']].values

song_table_data = song_data[['song_id','title','artist_id','year','duration']].values
song_table_data = list(song_table_data)
song_table_data

[array(['SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C',
        1982, 233.40363], dtype=object)]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [48]:
conn.rollback()

In [50]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [144]:
artist_data = list(song_data[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0])
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [133]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

ProgrammingError: can't adapt type 'numpy.ndarray'

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [55]:
log_files = get_files("data/log_data")
log_files

['/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-11-events.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-23-events.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-18-events.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-04-events.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-01-events.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-14-events.json',
 '/Users/veronica/Downloads/Udacity-Data-Engineering-Projects-master/Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-08-events

In [154]:
filepath = log_files

In [155]:
# open log file
df = df = pd.read_json(filepath, lines=True)

ValueError: Invalid file path or buffer object type: <class 'list'>

In [156]:
df = pd.DataFrame()
for file in filepath:
    with open(file,'r') as f:
        for line in f:
            try:
                json_data = json.loads(line)
                df= pd.concat([df,json_normalize(json_data)],axis=0,ignore_index=True)
            except json.JSONDecodeError as e:
                print(f'Error decoding json in file:{file}')
                print(f'Error message:{e}')

df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,None,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1540621059796.0,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1540621059796.0,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [157]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1540621059796.0,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1540809335796.0,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [80]:
type(df[['ts']])

pandas.core.frame.DataFrame

In [158]:
t = pd.to_datetime(df['ts'],unit='ms')
t.head()
type(t)


pandas.core.series.Series

In [159]:
df['ts'] = pd.to_datetime(df['ts'],unit='ms')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1540621059796.0,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1540809335796.0,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [87]:
#extract timestamp, hour, day, week of year, month, year, and weekday
def get_timeinfo(df,datetime):
    df_copy =df.copy()

    df_copy['hour'] = df_copy[datetime].dt.hour
    df_copy['day'] = df_copy[datetime].dt.day
    df_copy['weekofyear'] = df_copy[datetime].dt.weekofyear
    df_copy['month'] = df_copy[datetime].dt.month
    df_copy['year'] = df_copy[datetime].dt.year
    df_copy['weekday'] = df_copy[datetime].dt.weekday

    return df_copy
    
df_timeinfo = get_timeinfo(df,'ts')
df_timeinfo

# column_labels = ()

/var/folders/d5/p34ht6sj2cx51gt4sth4r59h0000gn/T/ipykernel_26286/944344928.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_copy['weekofyear'] = df_copy[datetime].dt.weekofyear


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,status,ts,userAgent,userId,hour,day,weekofyear,month,year,weekday
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2,11,45,11,2018,6
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2,11,45,11,2018,6
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2,11,45,11,2018,6
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,...,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,4,11,45,11,2018,6
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,...,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75,4,11,45,11,2018,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Motion City Soundtrack,Logged In,Stefany,F,1,White,205.26975,free,"Lubbock, TX",PUT,...,200,2018-11-24 20:45:53.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83,20,24,47,11,2018,5
8052,Ticanaf,Logged In,Stefany,F,2,White,2594.87302,free,"Lubbock, TX",PUT,...,200,2018-11-24 20:49:18.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83,20,24,47,11,2018,5
8053,The Far East Movement,Logged In,Ryan,M,0,Smith,228.20526,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,200,2018-11-24 22:15:50.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,22,24,47,11,2018,5
8054,the bird and the bee,Logged In,Ryan,M,1,Smith,189.51791,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,200,2018-11-24 22:19:38.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,22,24,47,11,2018,5


In [89]:
time_df = df_timeinfo[['ts','hour', 'day', 'weekofyear', 'month', 'year', 'weekday']]
time_df.head()

,ts,hour,day,weekofyear,month,year,weekday
0,2018-11-11 02:33:56.796,2,11,45,11,2018,6
1,2018-11-11 02:36:10.796,2,11,45,11,2018,6
2,2018-11-11 02:40:34.796,2,11,45,11,2018,6
4,2018-11-11 04:36:13.796,4,11,45,11,2018,6
5,2018-11-11 04:36:46.796,4,11,45,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [92]:
for i, row in time_df.iterrows():
    
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [93]:
user_df = df[['userId','firstName','lastName','gender','level']]
user_df

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
...,...,...,...,...,...
8051,83,Stefany,White,F,free
8052,83,Stefany,White,F,free
8053,26,Ryan,Smith,M,free
8054,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [94]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [95]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1541044398796.0,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1540621059796.0,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1540809335796.0,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [170]:
#Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

for index, row in df.iterrows():
    #print(row.song, row.artist, row.length)
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
        songplay_data = ( row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        #print(f'for {row.song},find id')
        print('find')

    else:
        songid, artistid = None, None
        #print(f'for {row.song},can not find id')
        pass
    # insert songplay record
    
    # cur.execute(songplay_table_insert, songplay_data)
    # conn.commit()

find


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [167]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.